# ADS 509 Module 3: Group Comparison 

The task of comparing two groups of text is fundamental to textual analysis. There are innumerable applications: survey respondents from different segments of customers, speeches by different political parties, words used in Tweets by different constituencies, etc. In this assignment you will build code to effect comparisons between groups of text data, using the ideas learned in reading and lecture.

This assignment asks you to analyze the lyrics for the two artists you selected in Module 1 and the Twitter descriptions pulled for Robyn and Cher. If the results from that pull were not to your liking, you are welcome to use the zipped data from the “Assignment Materials” section. Specifically, you are asked to do the following: 

* Read in the data, normalize the text, and tokenize it. When you tokenize your Twitter descriptions, keep hashtags and emojis in your token set. 
* Calculate descriptive statistics on the two sets of lyrics and compare the results. 
* For each of the four corpora, find the words that are unique to that corpus. 
* Build word clouds for all four corpora. 

Each one of the analyses has a section dedicated to it below. Before beginning the analysis there is a section for you to read in the data and do your cleaning (tokenization and normalization). 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


In [ ]:
# Use this space for any additional import statements you need



In [197]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.EMOJI_DATA : 
    for em in emoji.EMOJI_DATA[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """
    # Place your Module 2 solution here
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    if num_tokens > 0:
        lexical_diversity = num_unique_tokens/num_tokens
    else: lexical_diversity = 0
    num_characters = sum(len(token) for token in tokens)
    
    count = Counter(tokens)
    top_10 = count.most_common(10)

    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters, top_10])


    
def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if emoji.is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stop(tokens) :
    # modify this function to remove stopwords
    tokens = [word for word in tokens if word not in sw]
    return(tokens)
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    # Convert to lowercase and split into words
    # modify this function to return tokens
    words = str(text).lower().split()
    return(words)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

def clean(data, col, sw):
    text = ' '.join(data[col])
    # Remove punctuation
    text = re.sub(re.escape(str(tw_punct)), '', text)
    # Convert to lowercase and split into words
    words = str(text).lower().split()
    # Remove stopwords
    cleaned_tokens = [word for word in words if word not in sw]
    return cleaned_tokens

def sort(data, sort_col, comp_col):
    data = data[data[sort_col] > data[comp_col]]
    data = data.sort_values(by=sort_col, ascending=False).head(10)
    return data[['token', sort_col, comp_col]]

## Data Ingestion

Use this section to ingest your data into the data structures you plan to use. Typically this will be a dictionary or a pandas DataFrame.

In [6]:
# Feel free to use the below cells as an example or read in the data in a way you prefer

data_location = "/Users/mtc/ADS/ADS 509/M1 Results/" # change to your location if it is not in the same directory as your notebook
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

artist_files = {'cher':'cher_followers_data.txt',
                'robyn':'robynkonichiwa_followers_data.txt'}


In [7]:
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files['cher'],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = "cher"

In [8]:
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files['robyn'],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = "robyn"

twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)

In [13]:
# read in the lyrics here
lyrics_data = []

for artist in os.listdir(os.path.join(data_location,lyrics_folder)):
    path = os.path.join(data_location,lyrics_folder, artist)

    for file in os.listdir(path):

        song_path = path + '/' + file

        with open(song_path, 'r', encoding='utf-8') as s:
            lyrics = s.read()
            title = str(re.findall(r'".+"', lyrics))
            title = title[3:-3]
            lyrics_c = re.sub('^".+"', '', lyrics)
            lyrics_data.append({
                'artist' :artist,
                'song_title': title,
                'lyrics': lyrics_c
            })
lyrics_data = pd.DataFrame(lyrics_data)

## Tokenization and Normalization

In this next section, tokenize and normalize your data. We recommend the following cleaning. 

**Lyrics** 

* Remove song titles
* Casefold to lowercase
* Remove stopwords (optional)
* Remove punctuation
* Split on whitespace

Removal of stopwords is up to you. Your descriptive statistic comparison will be different if you include stopwords, though TF-IDF should still find interesting features for you. Note that we remove stopwords before removing punctuation because the stopword set includes punctuation.

**Twitter Descriptions** 

* Casefold to lowercase
* Remove stopwords
* Remove punctuation other than emojis or hashtags
* Split on whitespace

Removing stopwords seems sensible for the Twitter description data. Remember to leave in emojis and hashtags, since you analyze those. 

In [154]:
# apply the `pipeline` techniques from BTAP Ch 1 or 5

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

lyrics_data["tokens"] = lyrics_data["lyrics"].apply(prepare,pipeline=my_pipeline)
lyrics_data["num_tokens"] = lyrics_data["tokens"].map(len) 

twitter_data["tokens"] = twitter_data["description"].apply(prepare,pipeline=my_pipeline)
twitter_data["num_tokens"] = twitter_data["tokens"].map(len) 

In [155]:
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

Let's take a quick look at some descriptions with emojis.

In [156]:
twitter_data[twitter_data.has_emoji].sample(10)[["artist","description","tokens"]]

,artist,description,tokens
2548785,cher,"Venezolana 🌟 Aficionada a la fotografía, redes...","[venezolana, 🌟, aficionada, la, fotografía, re..."
1520851,cher,LoveDance💞,[lovedance💞]
2788691,cher,🎆,[🎆]
2318249,cher,"🌲 loving, rescue 🐶 advocate 💇🏼, cancer survivo...","[🌲, loving, rescue, 🐶, advocate, 💇🏼, cancer, s..."
1278749,cher,"#BLM, #climatechange, #LGBT 🌈 love animals am ...","[#blm, #climatechange, #lgbt, 🌈, love, animals..."
1353614,cher,keep an open mind 🍃,"[keep, open, mind, 🍃]"
676570,cher,#1 IASF stan. #1 Sweetener stan😌| ig- swtoutsold,"[#1, iasf, stan, #1, sweetener, stan😌, ig, swt..."
284418,cher,🏳️‍🌈 she/her - IG: sarah.smjw,"[🏳️‍🌈, sheher, ig, sarahsmjw]"
2804,cher,📖🤓,[📖🤓]
105523,cher,Fun loving caring one Ina million 🇯🇲♋️🇯🇲♋️ #MA...,"[fun, loving, caring, one, ina, million, 🇯🇲♋️🇯..."


With the data processed, we can now start work on the assignment questions. 

Q: What is one area of improvement to your tokenization that you could theoretically carry out? (No need to actually do it; let's not make perfect the enemy of good enough.)

A: I think the tokenization could benefit from removing non-English stopwords. At the top of this assignment, the stopwords pulled in were specifically the english ones. If a twitter description or set of lyrics has words that aren't in English, the current code cannot remove stopwords in those instances.

## Calculate descriptive statistics on the two sets of lyrics and compare the results. 


In [157]:
# your code here
r_lyrics = clean(lyrics_data[lyrics_data['artist']=='robyn'], 'lyrics', sw)
c_lyrics = clean(lyrics_data[lyrics_data['artist']=='cher'], 'lyrics', sw)

print('Lyrics- Robyn: Number of Tokens, Unique Tokens, Lexical Diversity, Number of Characters\n' + str(descriptive_stats(r_lyrics, verbose=True)))
print('Lyrics- Cher: Number of Tokens, Unique Tokens, Lexical Diversity, Number of Characters \n' + str(descriptive_stats(c_lyrics, verbose=True)))


Lyrics- Robyn: Number of Tokens, Unique Tokens, Lexical Diversity, Number of Characters
[14602, 2643, 0.18100260238323518, 73854, [('know', 297), ('love', 243), ('got', 240), ('like', 223), ('baby', 174), ('never', 148), ('dance', 143), ('get', 138), ('beat', 137), ('killing', 136)]]
Lyrics- Cher: Number of Tokens, Unique Tokens, Lexical Diversity, Number of Characters 
[33711, 4592, 0.13621666518347128, 168274, [('love', 893), ('know', 448), ('time', 299), ('see', 286), ('one', 267), ('like', 259), ('come', 248), ('take', 248), ('go', 247), ('never', 246)]]


Q: what observations do you make about these data? 

A: Cher has a higher number of characters, overall tokens, and a higher amount of unique tokens. This would make sense, since Cher's music career is quite long. Robyn's lyrics are more lexically diverse.


## Find tokens uniquely related to a corpus

Typically we would use TF-IDF to find unique tokens in documents. Unfortunately, we either have too few documents (if we view each data source as a single document) or too many (if we view each description as a separate document). In the latter case, our problem will be that descriptions tend to be short, so our matrix would be too sparse to support analysis. 

To avoid these problems, we will create a custom statistic to identify words that are uniquely related to each corpus. The idea is to find words that occur often in one corpus and infrequently in the other(s). Since corpora can be of different lengths, we will focus on the _concentration_ of tokens within a corpus. "Concentration" is simply the count of the token divided by the total corpus length. For instance, if a corpus had length 100,000 and a word appeared 1,000 times, then the concentration would be $\frac{1000}{100000} = 0.01$. If the same token had a concentration of $0.005$ in another corpus, then the concentration ratio would be $\frac{0.01}{0.005} = 2$. Very rare words can easily create infinite ratios, so you will also add a cutoff to your code so that a token must appear at least $n$ times for you to return it. 

An example of these calculations can be found in [this spreadsheet](https://docs.google.com/spreadsheets/d/1P87fkyslJhqXFnfYezNYrDrXp_GS8gwSATsZymv-9ms). Please don't hesitate to ask questions if this is confusing. 

In this section find 10 tokens for each of your four corpora that meet the following criteria: 

1. The token appears at least `n` times in all corpora
1. The tokens are in the top 10 for the highest ratio of appearances in a given corpora vs appearances in other corpora.

You will choose a cutoff for yourself based on the side of the corpus you're working with. If you're working with the Robyn-Cher corpora provided, `n=5` seems to perform reasonably well.

In [ ]:
# your code here
def get_count(tokens):
    count = Counter(tokens)
    df = pd.DataFrame.from_dict(count, orient = 'index').reset_index()
    df = df.rename(columns={'index':'token', 0:'count'})
    df['token']=df['token'].str.replace(',', '')
    df['concentration'] = df['count']/sum(df['count'])
    return(df[df['count']> 5])

In [194]:
r_corpus = get_count(r_lyrics)
r_corpus.rename(columns={'token':'token', 'count': 'count_r', 'concentration':'concentration_r'})

c_corpus = get_count(c_lyrics)
c_corpus.rename(columns={'token':'token', 'count': 'count_c', 'concentration':'concentration_c'})

comp = pd.merge(r_corpus, c_corpus, on=['token'], how='inner' )
comp['ratio_r_over_c'] = comp['concentration_x']/comp['concentration_y']
comp['ratio_c_over_r'] = comp['concentration_y']/comp['concentration_x']

In [ ]:
r_twitter = clean(twitter_data[twitter_data['artist'] == 'robyn'], "description", sw)
c_twitter = clean(twitter_data[twitter_data['artist'] == 'cher'], 'description', sw)

TypeError: sequence item 0: expected str instance, list found

In [198]:
print('Words with Higher Ratio in Robyn lyrics than Cher Lyrics \n', sort(comp, 'ratio_r_over_c', 'ratio_c_over_r'))
print('Words with Higher Ratio in Cher lyrics than Robyn Lyrics \n', sort(comp, 'ratio_c_over_r', 'ratio_r_over_c'))

Words with Higher Ratio in Robyn lyrics than Cher Lyrics 
      token  ratio_r_over_c  ratio_c_over_r
137   like       64.353796        0.015539
88   right       43.864471        0.022797
135   want       41.940590        0.023843
53    know       28.569622        0.035002
58   thing       24.240892        0.041253
14    beat       21.085728        0.047425
127   back       18.469251        0.054144
4     time       17.403717        0.057459
167  dance       15.006266        0.066639
261   stop       12.202898        0.081948
Words with Higher Ratio in Cher lyrics than Robyn Lyrics 
        token  ratio_c_over_r  ratio_r_over_c
276     know       32.342045        0.030920
355     love       32.233757        0.031023
284      say       10.890688        0.091822
254       go        9.726240        0.102815
294     find        9.529352        0.104939
206      man        7.599855        0.131581
238    right        6.822150        0.146581
225   enough        6.187891        0.161606
115 

Q: What are some observations about the top tokens? Do you notice any interesting items on the list? 

A: 

## Build word clouds for all four corpora. 

For building wordclouds, we'll follow exactly the code of the text. The code in this section can be found [here](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb). If you haven't already, you should absolutely clone the repository that accompanies the book. 


In [ ]:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

Q: What observations do you have about these (relatively straightforward) wordclouds? 

A: 